In [28]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
import datetime
from sklearn.model_selection import TimeSeriesSplit
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
#import xgboost as xgb
from sklearn.metrics import mean_squared_error
#from sklearn.preprocessing import RobustScaler
#from sklearn.model_selection import GridSearchCV

Load DataFrames and Preprocess

In [29]:
df = pd.read_csv("https://raw.githubusercontent.com/chencba/ContracetivesConsumptionPrediction/main/Train.csv")
product = pd.read_csv("https://raw.githubusercontent.com/chencba/ContracetivesConsumptionPrediction/main/product.csv")
df = df.merge(product, on = "product_code")

site = pd.read_csv("https://raw.githubusercontent.com/chencba/ContracetivesConsumptionPrediction/main/service_delivery_site_data.csv")
df = df.merge(site, on = "site_code")

df.drop(["site_region", "site_district"], axis = 1, inplace = True)

df.drop(["stock_stockout_days", "stock_adjustment", "site_latitude", "site_longitude", "product_name", "stock_ordered", "stock_end", "stock_initial", "average_monthly_consumption", "stock_received"], axis = 1, inplace = True)

df[:5]

,year,month,region,district,site_code,product_code,stock_distributed,product_type,site_type
0,2019,1,INDENIE-DJUABLIN,ABENGOUROU,C4001,AS27134,21,Injectable Contraceptive,Hospital
1,2019,2,INDENIE-DJUABLIN,ABENGOUROU,C4001,AS27134,0,Injectable Contraceptive,Hospital
2,2019,3,INDENIE-DJUABLIN,ABENGOUROU,C4001,AS27134,0,Injectable Contraceptive,Hospital
3,2019,4,INDENIE-DJUABLIN,ABENGOUROU,C4001,AS27134,2,Injectable Contraceptive,Hospital
4,2019,5,INDENIE-DJUABLIN,ABENGOUROU,C4001,AS27134,31,Injectable Contraceptive,Hospital


In [30]:
df["year"] = df["year"].astype(str)
df["month"] = df["month"].astype(str)

df["year_month"] = df["year"] + df["month"]
df["year_month"] = df["year_month"].apply(lambda x: datetime.datetime.strptime(x, "%Y%m"))

df["site_product"] = df["site_code"] + " X " + df["product_code"]

df[:5]

,year,month,region,district,site_code,product_code,stock_distributed,product_type,site_type,year_month,site_product
0,2019,1,INDENIE-DJUABLIN,ABENGOUROU,C4001,AS27134,21,Injectable Contraceptive,Hospital,2019-01-01,C4001 X AS27134
1,2019,2,INDENIE-DJUABLIN,ABENGOUROU,C4001,AS27134,0,Injectable Contraceptive,Hospital,2019-02-01,C4001 X AS27134
2,2019,3,INDENIE-DJUABLIN,ABENGOUROU,C4001,AS27134,0,Injectable Contraceptive,Hospital,2019-03-01,C4001 X AS27134
3,2019,4,INDENIE-DJUABLIN,ABENGOUROU,C4001,AS27134,2,Injectable Contraceptive,Hospital,2019-04-01,C4001 X AS27134
4,2019,5,INDENIE-DJUABLIN,ABENGOUROU,C4001,AS27134,31,Injectable Contraceptive,Hospital,2019-05-01,C4001 X AS27134


In [31]:
# Get index for train set
df[df["year_month"]<"4/1/2019"]

,year,month,region,district,site_code,product_code,stock_distributed,product_type,site_type,year_month,site_product
0,2019,1,INDENIE-DJUABLIN,ABENGOUROU,C4001,AS27134,21,Injectable Contraceptive,Hospital,2019-01-01,C4001 X AS27134
1,2019,2,INDENIE-DJUABLIN,ABENGOUROU,C4001,AS27134,0,Injectable Contraceptive,Hospital,2019-02-01,C4001 X AS27134
2,2019,3,INDENIE-DJUABLIN,ABENGOUROU,C4001,AS27134,0,Injectable Contraceptive,Hospital,2019-03-01,C4001 X AS27134
6,2018,1,INDENIE-DJUABLIN,ABENGOUROU,C4001,AS27134,13,Injectable Contraceptive,Hospital,2018-01-01,C4001 X AS27134
7,2018,2,INDENIE-DJUABLIN,ABENGOUROU,C4001,AS27134,5,Injectable Contraceptive,Hospital,2018-02-01,C4001 X AS27134
...,...,...,...,...,...,...,...,...,...,...,...
35733,2016,10,AGNEBY-TIASSA-ME,TIASSALE,C1009,AS27133,51,Injectable Contraceptive,Health Center,2016-10-01,C1009 X AS27133
35737,2019,3,AGNEBY-TIASSA-ME,TIASSALE,C1701,AS27138,0,Contraceptive Implant,Hospital,2019-03-01,C1701 X AS27138
35741,2019,3,AGNEBY-TIASSA-ME,TIASSALE,C1701,AS27133,0,Injectable Contraceptive,Hospital,2019-03-01,C1701 X AS27133
35745,2019,3,AGNEBY-TIASSA-ME,TIASSALE,C1701,AS21126,0,Injectable Contraceptive,Hospital,2019-03-01,C1701 X AS21126


In [32]:
# Get dummy variables
df = pd.get_dummies(df, columns = ["year", "month", "region", "district", "site_code", "product_code", "product_type", "site_type", "year_month", "site_product"], drop_first = True)
df[:5]

stock_distributed  year_2017  year_2018  year_2019  month_10  month_11  \
0                 21          0          0          1         0         0   
1                  0          0          0          1         0         0   
2                  0          0          0          1         0         0   
3                  2          0          0          1         0         0   
4                 31          0          0          1         0         0   

   month_12  month_2  month_3  month_4  month_5  month_6  month_7  month_8  \
0         0        0        0        0        0        0        0        0   
1         0        1        0        0        0        0        0        0   
2         0        0        1        0        0        0        0        0   
3         0        0        0        1        0        0        0        0   
4         0        0        0        0        1        0        0        0   

   month_9  region_ABIDJAN 2  region_AGNEBY-TIASSA-ME  region_BELIER  \
0        0                 0                        0              0   
1        0                 0                        0              0   
2        0                 0                        0              0   
3        0                 0                        0              0   
4        0                 0                        0              0   

   region_BOUNKANI-GONTOUGO  region_CAVALLY-GUEMON  region_GBEKE  \
0                         0                      0             0   
1                         0                      0             0   
2                         0                      0             0   
3                         0                      0             0   
4                         0                      0             0   

   region_GBOKLE-NAWA-SAN PEDRO  region_GOH  region_HAMBOL  \
0                             0           0              0   
1                             0           0              0   
2                             0           0              0   
3                             0           0              0   
4                             0           0              0   

   region_HAUT-SASSANDRA  region_INDENIE-DJUABLIN  \
0                      0                        1   
1                      0                        1   
2                      0                        1   
3                      0                        1   
4                      0                        1   

   region_KABADOUGOU-BAFING-FOLON  region_LOH-DJIBOUA  region_MARAHOUE  \
0                               0                   0                0   
1                               0                   0                0   
2                               0                   0                0   
3                               0                   0                0   
4                               0                   0                0   

   region_N'ZI-IFOU-MORONOU  region_PORO-TCHOLOGO-BAGOUE  region_SUD-COMOE  \
0                         0                            0                 0   
1                         0                            0                 0   
2                         0                            0                 0   
3                         0                            0                 0   
4                         0                            0                 0   

   region_TONKPI  region_WORODOUGOU-BERE  district_ABOBO-EST  \
0              0                       0                   0   
1              0                       0                   0   
2              0                       0                   0   
3              0                       0                   0   
4              0                       0                   0   

   district_ABOISSO  district_ADIAKE  district_ADJAME-PLATEAU-ATTECOUBE  \
0                 0                0                                  0   
1                 0                0                                  0   
2            

Train & Test Split

In [33]:
train = df.iloc[:28602,:]
test = df.iloc[28602:,:]

In [34]:
X = train.drop("stock_distributed", axis = 1)
y = train["stock_distributed"]

Cross Validation

In [35]:
mses = []

cv = TimeSeriesSplit(n_splits = 5, max_train_size = 15000, test_size = 4000)

for train_index, val_index in cv.split(X):
     print("Train:", train_index, "Validation:", val_index)
     X_train, X_val = X.loc[train_index, :], X.loc[val_index, :]
     y_train, y_val = y[train_index], y[val_index]
     
     #transformer = RobustScaler().fit(X_train)
     #X_train = transformer.transform(X_train)
     #X_val = transformer.transform(X_val)

     model = RandomForestRegressor(max_depth=2)
     # min_samples_split=2, random_state=44, max_depth=2, criterion='absolute_error'
     # RandomForestRegressor() LinearRegression()
     model.fit(X_train, y_train)
     pred = model.predict(X_val)
     
     mse = mean_squared_error(y_val, pred)
     mses.append(mse)
     
print("MSE's:", mses)

average_mse = np.mean(mses)
std_mse = np.std(mses)

print("AVG MSE: ", average_mse)
print("STD MSE: ", std_mse)

Train: [   0    1    2 ... 8599 8600 8601] Validation: [ 8602  8603  8604 ... 12599 12600 12601]
Train: [    0     1     2 ... 12599 12600 12601] Validation: [12602 12603 12604 ... 16599 16600 16601]
Train: [ 1602  1603  1604 ... 16599 16600 16601] Validation: [16602 16603 16604 ... 20599 20600 20601]
Train: [ 5602  5603  5604 ... 20599 20600 20601] Validation: [20602 20603 20604 ... 24599 24600 24601]
Train: [ 9602  9603  9604 ... 24599 24600 24601] Validation: [24602 24603 24604 ... 28599 28600 28601]
MSE's: [3248.0078178206722, 1793.2918379217137, 849.5093447569187, 1748.034945848798, 1188.133433324738]
AVG MSE:  1765.3954759345681
STD MSE:  821.133097914292


Get important features

In [36]:
# Get all feature names
feature_list = list(X.columns)

# Get numerical feature importances
importances = list(model.feature_importances_)

# List of tuples with variable and importance
feature_importances = [(feature, round(importance, 2)) for feature, importance in zip(feature_list, importances)]

# Sort the feature importances by most important first
feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)

# Print out the features and importances 
for i in feature_importances[:20]:
  print(i)

('site_product_C1088 X AS27134', 0.32)
('site_product_C2064 X AS27000', 0.25)
('site_product_C2006 X AS27000', 0.21)
('site_product_C1088 X AS27133', 0.1)
('year_month_2016-02-01 00:00:00', 0.03)
('year_month_2018-07-01 00:00:00', 0.02)
('year_2017', 0.01)
('year_2018', 0.01)
('year_month_2016-06-01 00:00:00', 0.01)
('year_month_2016-10-01 00:00:00', 0.01)
('year_month_2017-01-01 00:00:00', 0.01)
('year_2019', 0.0)
('month_10', 0.0)
('month_11', 0.0)
('month_12', 0.0)
('month_2', 0.0)
('month_3', 0.0)
('month_4', 0.0)
('month_5', 0.0)
('month_6', 0.0)


MSE on test set

In [37]:
X_test = test.drop("stock_distributed", axis = 1)
y_test = test["stock_distributed"]

mean_squared_error(model.predict(X_test), y_test)

1994.7767176029568